In [ ]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
import networkx as nx
from scipy.spatial.distance import cdist
import sklearn
from sklearn.cluster import Birch, KMeans, MeanShift, DBSCAN, SpectralClustering
from sklearn.decomposition import TruncatedSVD
from normalized_cut import normalized_cut

from point_cloud_utils import get_pcd, transform_pcd
from aggregate_pointcloud import aggregate_pointcloud
from reproject_merged_pointcloud import reproject_points_to_label, merge_associations, merge_features
from visualization_utils import generate_random_colors

Here we define the dataset depending on kitti sequence!

In [ ]:
DATASET_PATH = os.path.join('/Users/laurenzheidrich/Downloads/','fused_dataset')
SEQUENCE_NUM = 7

config_filtered = KittiOdometryDatasetConfig(
    cache=True,
    dataset_path=DATASET_PATH,
    correct_scan_calibration=True,
    filters=FilterList(
        [
            KittiGTMovingObjectFilter(
                os.path.join(
                    DATASET_PATH,
                    "sequences",
                    "%.2d" % SEQUENCE_NUM,
                    "labels",
                )
            ),
            RangeFilter(2.5, 120),
            ApplyPose(),
        ]
    ),
)

dataset = KittiOdometryDataset(config_filtered, SEQUENCE_NUM)

Now we read in the point cloud and the left and right image of the stereo camera. If labels for those images are available they can be read in, too!

In [ ]:
ind_start = 58
ind_end = 61
upper_threshold = 26 #meters
lower_threshold = 6 #meters
voxel_size = 0.35

pcd_aggregated, T_pcd = aggregate_pointcloud(dataset, ind_start, ind_end, clip_to_imageframe=True)

pcd_cut = pcd_aggregated
pcd_cut = pcd_cut.select_by_index(np.where(np.asarray(pcd_cut.points)[:,0] > lower_threshold)[0])
pcd_cut = pcd_cut.select_by_index(np.where(np.asarray(pcd_cut.points)[:,0] < upper_threshold)[0])

pcd = pcd_cut.voxel_down_sample(voxel_size=voxel_size)

num_points = np.asarray(pcd.points).shape[0]
print("num points: ", num_points)

No we can to the point to label reprojection for each timestep and merge them in a list

In [ ]:
cams = ["cam2", "cam3"]
cam_ind = 0
point_to_label_reprojections = []
point_to_dinov2_feature_reprojections = []
concatenated_tarl_points = np.zeros((0, 3))
concatenated_tarl_features = np.zeros((0, 96))

for points_index in range(ind_start-1, ind_end+1):
	# Load the SAM label
	label_PIL = dataset.get_sam_label(cams[cam_ind], points_index)
	label = cv2.cvtColor(np.array(label_PIL), cv2.COLOR_RGB2BGR)

	# Load the DINOV2 feature map
	dinov2_feature_map = dataset.get_dinov2_features(cams[cam_ind], points_index)
	dinov2_feature_map_zoomed = scipy.ndimage.zoom(dinov2_feature_map, (label.shape[0] / dinov2_feature_map.shape[0], label.shape[1] / dinov2_feature_map.shape[1], 1), order=1)

	# Load the TARL features
	coords, _, point_features, cluster_indices = dataset.get_tarl_features(points_index)
	
	# Load the calibration matrices
	T_lidar2world = dataset.get_pose(points_index)
	T_world2lidar = np.linalg.inv(dataset.get_pose(points_index))
	T_lidar2cam, K = dataset.get_calibration_matrices(cams[cam_ind])
	T_world2cam = T_lidar2cam @ T_world2lidar
	
	# Compute the SAM label reporjections
	point_to_label_reprojections.append(reproject_points_to_label(np.array(pcd.points), T_pcd, label, T_world2cam, K, hidden_point_removal=True))

	# Compute the DINOV2 feature map reprojections
	point_to_dinov2_feature_reprojections.append(reproject_points_to_label(np.array(pcd.points), T_pcd, dinov2_feature_map_zoomed, T_world2cam, K, hidden_point_removal=True, label_is_color=False))

	# Compute the TARL features 
	T_local2global_pcd = np.linalg.inv(T_pcd) @ T_lidar2world
	coords = transform_pcd(coords, T_local2global_pcd)
	mask = np.where((coords[:,0] < upper_threshold) & (coords[:,0] > lower_threshold))
	coords, point_features, cluster_indices = coords[mask], point_features[mask], cluster_indices[mask]
	concatenated_tarl_points = np.concatenate((concatenated_tarl_points, coords))
	concatenated_tarl_features = np.concatenate((concatenated_tarl_features, point_features))

The TARL features now need to be projected on the original downsampled PCD. This is done by computing the mean over radius-based nearest neigbors

In [ ]:
tarl_pcd = get_pcd(concatenated_tarl_points)
tarl_tree = o3d.geometry.KDTreeFlann(tarl_pcd)

tarl_features = np.zeros((num_points, 96))

i=0
for point in np.asarray(pcd.points):
    [_, idx, _] = tarl_tree.search_radius_vector_3d(point, voxel_size)
    features_in_voxel = concatenated_tarl_features[idx]
    tarl_features[i,:] = np.mean(features_in_voxel, axis=0)
    i+=1

From the merged list we can now build the association matrix

In [ ]:
sam_label_matrix = merge_associations(point_to_label_reprojections, len(pcd.points))

In [ ]:
dinov2_feature_matrix = merge_features(point_to_dinov2_feature_reprojections, len(pcd.points))

Here we define important parameters and build the point-to-point distance matrix

In [ ]:
proximity_threshold = 2 # meters that points can be apart from each other and still be considered neighbors
alpha = 2.3 # weight of the spatial proximity term  2.3
beta = 1.1 #1.1 # weight of the label similarity term 1.1
gamma = 0.1 # weight of the dinov2 feature similarity term 1.5
theta = 1.4 # weight of the tarl feature similarity term
points = np.asarray(pcd.points)
dist_matrix = cdist(points, points)
dinov2_distance = cdist(dinov2_feature_matrix, dinov2_feature_matrix)
tarl_distance = cdist(tarl_features, tarl_features)

Now we can build the graph with nodes and edges! First we build the matrix that measures the point-to-point distance in the feature space

In [ ]:
mask = np.where(dist_matrix <= proximity_threshold)
not_ordered_dist_mat_emb = np.sum(np.abs(sam_label_matrix[mask[0]] - sam_label_matrix[mask[1]]), axis=1)

embedding_dist_matrix = np.zeros(dist_matrix.shape)
for k, (i, j) in enumerate(zip(*mask)):
    embedding_dist_matrix[i, j] = not_ordered_dist_mat_emb[k][0, 0]

mask = np.where(dist_matrix <= proximity_threshold, 1, 0)

label_distance = (mask - np.eye(num_points)) * np.exp(-beta * embedding_dist_matrix)

Now we build the adjacency matrix amd remove all isolated points

In [ ]:
A = np.exp(-alpha * (mask * dist_matrix)) * label_distance * np.exp(-theta * (mask * tarl_distance))
isolated_mask = ~np.all(A == 0, axis=1)
A = A[isolated_mask][:, isolated_mask]
pcd = pcd.select_by_index(np.where(isolated_mask == True)[0])
print(num_points - np.asarray(pcd.points).shape[0], "isolated points removed")
num_points = np.asarray(pcd.points).shape[0]

Here we can check for the number of connected components and number of isolated nodes in the constructed graph

In [ ]:
G = nx.from_numpy_array(A)

isolated_nodes = [node for node, degree in G.degree() if degree == 0]
print("There are", len(isolated_nodes), "isolated nodes")
print(isolated_nodes)

connected_components = list(nx.connected_components(G))
print("There are", len(connected_components), "connected components")

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges()) 

Now we can perform normalized cuts and save the cut graph

In [ ]:
grouped_labels = normalized_cut(A, np.arange(num_points), T = 0.00025)

In [ ]:
print(len(grouped_labels))

Now we visualize the different components and save them to a point cloud file

In [ ]:
print("There are", len(grouped_labels), "cut regions")

random_colors = generate_random_colors(600)

pcd_color = np.zeros((num_points, 3))

for i, s in enumerate(grouped_labels):
    for j in s:
        pcd_color[j] = np.array(random_colors[i]) / 255

pcd.colors = o3d.utility.Vector3dVector(pcd_color)

#o3d.io.write_point_cloud("pcd_merge_clustered.pcd", pcd, write_ascii=False, compressed=False, print_progress=False)
#o3d.visualization.draw_geometries([pcd])

With this method, we can perform the nearest point labeling on the big point cloud, if the clustering beforehand was done on a sampled point cloud

In [ ]:
pcd_cut.paint_uniform_color([0, 0, 0])
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

i=0
for point in np.asarray(pcd_cut.points):
    [_, idx, _] = pcd_tree.search_knn_vector_3d(point, 1)
    np.asarray(pcd_cut.colors)[i,:] = np.asarray(pcd.colors)[idx[0], :]
    i+=1

o3d.io.write_point_cloud("pcd_merge_clustered.pcd", pcd_cut, write_ascii=False, compressed=False, print_progress=False)
#o3d.visualization.draw_geometries([pcd_cut])

We can also perform spectral clustering instead of normalized cuts:

In [ ]:
n_clusters = 55
A = nx.adjacency_matrix(G).toarray()
sc = SpectralClustering(n_clusters=n_clusters, affinity="precomputed", n_init=100, assign_labels="discretize")
labels = sc.fit_predict(A)  

In [ ]:
colorspace = plt.cm.rainbow(np.linspace(0, 1, n_clusters))[:, :3]
colors = [colorspace[i] for i in labels]
pcd.colors = o3d.utility.Vector3dVector(np.array(colors))
#o3d.io.write_point_cloud("pcd_merge_clustered.pcd", pcd, write_ascii=False, compressed=False, print_progress=False)

Again, we can perform the nearest point labeling on the big point cloud, if the clustering beforehand was done on a sampled point cloud

In [ ]:
pcd_cut.paint_uniform_color([0, 0, 0])
pcd_tree = o3d.geometry.KDTreeFlann(pcd)

i=0
for point in np.asarray(pcd_cut.points):
    [_, idx, _] = pcd_tree.search_knn_vector_3d(point, 1)
    np.asarray(pcd_cut.colors)[i,:] = np.asarray(pcd.colors)[idx[0], :]
    i+=1

o3d.io.write_point_cloud("pcd_merge_clustered.pcd", pcd_cut, write_ascii=False, compressed=False, print_progress=False)

We can also perform other clustering methods of the feature matrix, that do not take the graph as input:

In [ ]:
n_components = 25
tSVD = TruncatedSVD(n_components=25)
transformed_data = tSVD.fit_transform(association_matrix)

In [ ]:
# Birch is fastest, results are also quite nice
birch_model = Birch(threshold=0.1, n_clusters=25)
birch_model.fit(transformed_data)
labels = birch_model.predict(transformed_data)

# kmeans is quite fast, results look okay, some noise
#kmeans = KMeans(n_clusters=80)
#labels = kmeans.fit_predict(transformed_data)

# DBSCAN doesn't really produce great results and takes really long
#dbscan = DBSCAN(eps=0.5, min_samples=100)
#labels = dbscan.fit_predict(transformed_data)

# Meanshift takes way too long
#meanshift = MeanShift()
#labels = meanshift.fit_predict(transformed_data)

In [ ]:
print("Cluster Assignments:")
print(len(set(labels)))

In [ ]:
colorspace = plt.cm.rainbow(np.linspace(0, 1, len(set(labels))))[:, :3]
colors = [colorspace[i] for i in labels]
pcd.colors = o3d.utility.Vector3dVector(np.array(colors))
o3d.io.write_point_cloud("pcd_merge_clustered.pcd", pcd, write_ascii=False, compressed=False, print_progress=False)